# Best Pittsburgh Neighboorhood

## Team Stuff

#### Introduction:

Datasets used:
- [Arrests.csv](https://data.wprdc.org/dataset/non-traffic-citations)
- [educational-attainment.csv](https://data.wprdc.org/dataset/pittsburgh-american-community-survey-2015-miscellaneous-data/resource/12535b2e-6180-4cdf-b7d8-ec5294259e49)
- [GunSeizures.csv](https://data.wprdc.org/dataset/pbp-fire-arm-seizures)